# JIT Fallback

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/experts/zh_cn/network/mindspore_jit_fallback.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/experts/zh_cn/network/mindspore_jit_fallback.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/experts/source_zh_cn/network/jit_fallback.ipynb)

在MindSpore静态图模式下，用户编写程序时需要遵循MindSpore[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/note/static_graph_syntax_support.html)，语法使用存在约束限制。而在动态图模式下，Python脚本代码会根据Python语法进行执行，用户可以使用任意Python语法。可以看出，静态图和动态图的语法约束限制是不同的。

JIT Fallback是从静态图的角度出发考虑静态图和动态图的统一。通过JIT Fallback特性，静态图可以支持尽量多的动态图语法，使得静态图提供接近动态图的语法使用体验，从而实现动静统一。为了便于用户选择是否使用JIT Fallback特性的能力，提供了JIT语法支持级别选项`jit_syntax_level`，其值必须在[STRICT(0)，COMPATIBLE(1)，LAX(2)]范围内，默认值为LAX(2)。全部级别都支持所有后端。

STRICT(0): 仅支持基础语法，且执行性能最佳。
COMPATIBLE(1): 除支持基础语法外，还支持更多语法，如dict，list和scalar的操作等。
LAX(2): 最大程度地兼容Python所有语法。执行性能可能会受影响，不是最佳。

下面主要介绍JIT Fallback的支持范围和使用须知，以便您可以更有效地使用JIT Fallback功能。

## 支持范围

JIT Fallback特性还在持续完善中，下面列举出当前通过该特性已经支持的静态图编译语法。

## 创建和使用Tensor

JIT Fallback支持在静态图模式下创建和使用[Tensor](https://www.mindspore.cn/docs/zh-CN/master/api_python/mindspore/mindspore.Tensor.html)。

代码用例如下，用例中的`Tensor(1, dtype=mstype.int32)`是通过JIT Fallback支持的。

In [7]:
import mindspore.nn as nn
import mindspore as ms

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()

    def construct(self):
        return ms.Tensor(1, dtype=ms.int32)

ms.set_context(mode=ms.GRAPH_MODE)

net = Net()
print(net())

1

上述例子，使用了Tensor类接口创建Tensor，有些情况下可能会需要创建运行时的Tensor，此时既可以采用上述类ms.Tensor接口来创建Tensor，也可以采用 [tensor函数接口](https://www.mindspore.cn/docs/zh-CN/master/api_python/mindspore/mindspore.tensor.html#mindspore.tensor)来创建Tensor，代码用例如下。

In [17]:
import mindspore as ms
import mindspore.nn as nn

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()

    @ms.jit
    def construct(self, x):
        return ms.tensor(x.asnumpy(), dtype=ms.float32)

ms.set_context(mode=ms.GRAPH_MODE)
net = Net()
x = ms.Tensor(1, dtype=ms.int32)
print(net(x))


1.0


## Annotation 标记

对于Runtime时的JIT Fallback支持，会产生一些无法被类型推导出的节点，这种类型称为Any类型。因为该类型无法在编译时推导出正确的类型，所以这种Any将会以一种默认最大精度Float64进行运算，防止其精度丢失。为了能更好的优化相关性能，需要减少Any的产生。当用户可以明确知道当前通过JIT Fallback支持的语句会产生什么类型的时候，我们推荐使用`Annotation @jit.typing:`的方式进行指定对应Python语句类型，从而确定解释节点的类型避免Any类型的生成。

例如，上述例子Tensor类和tensor接口的区别就在于在tensor接口内部运用了Annotation机制。当tensor函数的dtype确定时，函数内部会利用Annotation指定输出类型从而避免Any类型的产生。Annotation的使用只需要在对应Python语句上面或者后面加上注释 `# @jit.typing: () -> tensor_type[float32]` 即可，其中 `->` 后面的 `tensor_type[float32]` 指示了被注释的语句输出类型。

代码用例如下。

In [4]:
import mindspore as ms
import mindspore.nn as nn
from mindspore import ops, Tensor

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.abs = ops.Abs()

    @ms.jit
    def construct(self, x, y):
        y1 = ms.tensor(x.asnumpy() + y.asnumpy(), dtype=ms.float32)
        y2 = ms.Tensor(x.asnumpy() + y.asnumpy(), dtype=ms.float32) # @jit.typing: () -> tensor_type[float32]
        y3 = Tensor(x.asnumpy() + y.asnumpy())
        y4 = Tensor(x.asnumpy() + y.asnumpy(), dtype=ms.float32)
        return self.abs(y1), self.abs(y2), self.abs(y3), self.abs(y4)

ms.set_context(mode=ms.GRAPH_MODE)
net = Net()
x = ms.Tensor(-1, dtype=ms.int32)
y = ms.Tensor(-1, dtype=ms.float32)
y1, y2, y3, y4 = net(x, y)

print(f"y1 value is {y1}, dtype is {y1.dtype}")
print(f"y2 value is {y2}, dtype is {y2.dtype}")
print(f"y3 value is {y3}, dtype is {y3.dtype}")
print(f"y4 value is {y4}, dtype is {y4.dtype}")

y1 value is 2.0, dtype is Float32
y2 value is 2.0, dtype is Float32
y3 value is 2.0, dtype is Float64
y4 value is 2.0, dtype is Float64


上述例子，可以看到利用JIT Fallback Runtime创建了Tensor的相关区别。对于y3、y4，因为Tensor类没有增加Annotation指示，y3、y4没有办法推出正确的类型，导致只能按照最高精度Float64进行运算。
对于y2，由于创建Tensor时，通过Annotation指定了JIT Fallback的对应类型，使得其类型可以按照指定类型进行运算。
对于y1，由于使用了tensor函数接口创建Tensor，传入的dtype参数作为Annotation的指定类型，所以也避免了Any类型的产生。

## 调用第三方库

JIT Fallback支持在静态图模式下调用第三方库的对象和方法。

需要说明的是，对于具有返回值的方法，需要使用变量来保存其结果，否则可能出现报错。这个用法将在后续版本中支持。

调用第三方库的代码用例如下。用例调用了NumPy第三方库，其中`np.array([1, 2, 3])`和`np.array([4, 5, 6])`是通过JIT Fallback支持的。


In [8]:
import numpy as np
import mindspore as ms
import mindspore.nn as nn

# pylint: disable= W0235
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()

    def construct(self):
        a = np.array([1, 2, 3])
        b = np.array([4, 5, 6])
        c = a + b
        return ms.Tensor(c)

ms.set_context(mode=ms.GRAPH_MODE)
net = Net()
print(net())

[5 7 9]

## 使用Python原生的print打印

JIT Fallback支持在静态图模式下使用Python原生的print来打印常量，它与[Print算子](https://www.mindspore.cn/docs/zh-CN/master/api_python/ops/mindspore.ops.Print.html)打印信息的时机有所不同。Python原生print是在编译过程中触发打印（编译时阶段打印），而Print算子是需要图编译完成后，下发到设备端运行才打印（运行时阶段打印）。

为了便于理解，举例如下。tensor_sum涉及Tensor相加，即运行时阶段才能得到结果，在调用print时，实际调用的是静态图模式中的Print算子，参考[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/note/static_graph_syntax_support.html)。而np_num是由两个NumPy常量相加得到的结果，即通过JIT Fallback支持的用法，因此在调用print时，使用的是Python原生print。由于两者的打印时机不同，最终导致显示np_sum在tensor_sum之前，即通过JIT Fallback支持的Python原生print的打印结果会在Print算子之前。


In [9]:
import numpy as np
import mindspore as ms
import mindspore.nn as nn

# pylint: disable= W0235
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()

    def construct(self):
        x = ms.Tensor(np.array([1, 2, 3, 4, 5]))
        y = ms.Tensor(np.array([1, 2, 3, 4, 5]))
        tensor_sum = x + y
        print("tensor_sum: ", tensor_sum)
        x = np.array([1, 2, 3, 4, 5])
        y = np.array([1, 2, 3, 4, 5])
        np_sum = x + y
        print("np_sum: ", np_sum)
        return tensor_sum, ms.Tensor(np_sum)

ms.set_context(mode=ms.GRAPH_MODE)
net = Net()
net()

np_sum: [2 4 6 8 10]
tensor_sum: (2, 4, 6, 8, 10)

## 使用raise和assert

JIT Fallback支持在静态图模式下使用raise和assert。

支持使用raise，用例如下：

```python

import mindspore.nn as nn
import mindspore as ms

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()

    def construct(self, x, y):
        if x <= y:
            raise ValueError("x should be greater than y.")
        else:
            x += 1
        return x

ms.set_context(mode=ms.GRAPH_MODE)
net = Net()
net(ms.Tensor(-2), ms.Tensor(-1))

```

输出结果:

```text

ValueError: x should be greater than y.

```

支持使用assert，用例如下：

```python

import mindspore.nn as nn
import mindspore as ms

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()

    def construct(self, x):
        assert x in [2, 3, 4]
        return x

ms.set_context(mode=ms.GRAPH_MODE)
net = Net()
net(ms.Tensor(-1))

```

输出结果中正常出现:

```text

AssertionError.

```

## 调用Python内置函数

MindSpore在静态图模式下已经支持了一些Python内置函数，包括但不限于len、isinstance、map、zip等，详情请参考[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/note/static_graph_syntax_support.html)。通过JIT Fallback，可以在常量场景中支持更多的Python内置函数的用法。下面简单举例支持的部分Python内置函数。

### dict()

功能：用于创建一个字典。

有效输入：字典的 Key 只支持 String 类型，Value 只支持常量，不支持自定义类。

暂不支持对 `dict()` 创建的字典进行循环遍历，包括 `dict.keys()`、`dict.values()`、`dict.items()`。

代码用例如下：


In [10]:
import mindspore as ms

@ms.jit
def func():
    a = dict()                                          # 创建空字典
    b = dict(a='a', b='b', t='t')                       # 传入关键字
    c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))   # 映射函数方式来构造字典
    d = dict([('one', 1), ('two', 2), ('three', 3)])    # 可迭代对象方式来构造字典
    return a, b, c, d

a, b, c, d = func()
print("a: ", a)
print("b: ", b)
print("c: ", c)
print("d: ", d)


a: {}
b: {'a': 'a', 'b': 'b', 't': 't'}
c: {'one': 1, 'two': 2, 'three': 3}
d: {'one': 1, 'two': 2, 'three': 3}


### type()

功能：输出入参的类型。

有效输入：Number、list、tuple、dict、np.array、常量Tensor。

代码用例如下：


In [11]:
import numpy as np
import mindspore as ms

@ms.jit
def func():
    a = type(1)
    b = type(1.0)
    c = type([1, 2, 3])
    d = type((1, 2, 3))
    e = type({'a': 1, 'b': 2})
    f = type(np.array([1, 2, 3]))
    g = type(ms.Tensor([1, 2, 3]))
    return a, b, c, d, e, f, g

a, b, c, d, e, f, g = func()
print("a: ", a)
print("b: ", b)
print("c: ", c)
print("d: ", d)
print("e: ", e)
print("f: ", f)
print("g: ", g)


a: <class 'int'>
b: <class 'float'>
c: <class 'list'>
d: <class 'tuple'>
e: <class 'dict'>
f: <class 'numpy.ndarray'>
g: <class 'mindspore.common.tensor.Tensor'>


> type作为Python的原生函数还有另外一种使用方法，即type(name, bases, dict)返回name类型的类对象，由于该用法应用场景较少，因此暂不支持。

## 支持控制流

为了提高Python标准语法支持度，实现动静统一，通过JIT Fallback实现控制流语句的使用。控制流语句是指if、for、while等流程控制语句。JIT Fallback特性已经支持在静态图模式下创建和使用Tensor，支持调用Numpy等第三方库创建使用常量和变量以及支持部分Python内置函数。理论上，通过JIT Fallback支持的语法，在控制流场景中也支持。
代码用例如下：


In [ ]:
import numpy as np
import mindspore as ms

@ms.jit
def func():
    x = np.array(1)
    if x <= 1:
        x += 1
    return ms.Tensor(x)

res = func()
print("res: ", res)


res: 2


## 支持运行时(Runtime)阶段的JIT Fallback

JIT Fallback处理不支持的语法表达式时，将会生成相应的节点，常量会在编译时阶段推导出值，否则这些节点将传递到后端运行时，在后端通过Python的能力执行得到结果。示例代码如下，`np.add(x, y)`会生成相应节点，作为函数的返回值将会传递到运行时，当前已支持部分场景下的运行时阶段的JIT Fallback。


In [ ]:
import numpy as np
import mindspore as ms

@ms.jit
def test_np_add():
    x = np.array([1, 2, 3, 4, 5])
    y = np.array([1, 2, 3, 4, 5])
    return np.add(x, y)

np_add_res = test_np_add()
print(np_add_res)

[ 2  4  6  8  10]


## 顶层图支持返回list、dict、scalar、none等基础类型

### 顶层图支持返回list


In [ ]:
import mindspore as ms

@ms.jit
def test_return_list():
    return [1, "a", True, None, ms.Tensor([2])]

res = test_return_list()
print(res)

[1, "a", True, None, Tensor([2])]


### 顶层图支持返回dict

In [ ]:
import mindspore as ms

@ms.jit
def test_return_dict():
    x = {'a': 1, 'b': 2}
    y = x.get('a')
    y_tensor = ms.Tensor([y])
    z = dict(a=y_tensor)
    return z

res = test_return_dict()
print(res)

{'a': ms.Tensor(np.array(1), ms.int64)}


### 顶层图支持返回scalar

In [ ]:
import mindspore as ms

@ms.jit
def test_return_scalar(x, y):
    return x + y

res = test_return_scalar(ms.mutable(1), ms.mutable(2))
print(res)

3


### 顶层图支持返回None

In [ ]:
import mindspore as ms

@ms.jit
def test_return_none():
    return 1, "a", None

res = test_return_none()
print(res)

(1, 'a', None)


## 使用须知

在使用JIT Fallback时，请注意以下几点：

1.JIT Fallback对标动态图的支持能力，须在动态图语法范围内，包括但不限于数据类型等。

2.当前常量控制流场景中暂不支持对Numpy Array数据的取下标赋值，错误的代码用例如下：

```python

import numpy as np
import mindspore as ms

@ms.jit
def func():
    x = np.array([1, 2, 3])
    x[0] += 1
    return ms.Tensor(x)

res = func()
print("res: ", res)

```

报错信息如下:

```text

RuntimeError: For operation 'setitem', current input arguments types are <External, Number, Number>. The 1-th argument type 'External' is not supported now.

```